In [ ]:
import pandas as pd
import mysql.connector
from DFStruct import *

# Function to map pandas dtypes to MySQL types
def map_dtype_to_mysql(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_string_dtype(dtype):
        return 'VARCHAR(255)'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

database_name = 'DB_football'
try:
    # Create Connection
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="_!0qo__trL-mY##3oSoiI#93"
    )
    cursor = connection.cursor()
    cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    cursor.execute(f"USE {database_name}")
    # Drop existing tables
    tables = ['shots', 'matches', 'players', 'teams']
    for table in tables:
        cursor.execute(f"DROP TABLE IF EXISTS {table}")
    # Get columns to drop and read CSVs
    shots_drop_columns, matches_drop_columns, players_drop_columns, teams_drop_columns = get_drop_columns()
    df_shots, df_matches, df_players, df_teams = read_all_csvs(shots_drop_columns, matches_drop_columns, players_drop_columns, teams_drop_columns)
    df_list = [df_shots, df_matches, df_players, df_teams]
    # Function to create table dynamically
    def create_table_from_df(df, table_name):
        column_definitions = []
        for col, dtype in zip(df.columns, df.dtypes):
            mysql_type = map_dtype_to_mysql(dtype)
            column_definitions.append(f"`{col}` {mysql_type}")
        
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (\n" + ",\n".join(column_definitions) + "\n);"
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created successfully!")
    # Create tables for each DataFrame
    for df, table in zip(df_list, tables):
        create_table_from_df(df, table)
    # Insert data into tables
    def insert_data_from_df(cursor, df, table_name):
        for _, row in df.iterrows():
            columns = ', '.join(df.columns)
            values = ', '.join([f"'{str(value)}'" for value in row])
            insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values});"
            cursor.execute(insert_query)
        print(f"Data inserted into {table_name} successfully!")
    for df, table in zip(df_list, tables):
        insert_data_from_df(cursor, df, table)
    # Commit the changes
    connection.commit()
    print("All tables created and data inserted successfully!")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    connection.rollback()

